# Session analysis

In [2]:
import pandas as pd

from mna.sessions.eye_session import process_session_eye
from mna.sessions.eeg_session import process_session_eeg
from mna.sessions.motor_session import process_session_motor
from mna.sessions.ecg_session import process_session_ecg

In [3]:
import matplotlib
matplotlib.use('Agg')

# 1. Read a RN App, converted pkl file, and create the metadata and data structure

In [5]:
from os import listdir
from os.path import isfile, join
from mna.utils.rnapp_data_format import read_all_lslpresets, return_metadata_from_name, event_data_from_data
import pickle

In [4]:
data_dir = "/Volumes/GoogleDrive/My Drive/Research Work/BCIFlow/Pkl_Recordings/"
lsl_dir = "../mna/LSLPresets/"
onlyfiles = [f for f in listdir(data_dir) if isfile(join(data_dir, f)) and '.pkl' in f]
input_path = data_dir + onlyfiles[4] # pick a random file
print(f"input_path {input_path}")
metadata_jsons = read_all_lslpresets(path_to_jsonfiles=lsl_dir)
with open(input_path, 'rb') as handle:
    rns_data = pickle.load(handle)
    
## Add metadata to data

for key in rns_data.keys():
    rns_data[key].append(return_metadata_from_name(key, metadata_jsons))

input_path /Volumes/GoogleDrive/My Drive/Research Work/BCIFlow/Pkl_Recordings/06_15_2022_10_38_35-Exp_myexperiment-Sbj_02-Ssn_02.dats.pkl


# 2. Create new events (trial start etc.)

In [5]:
event_df = event_data_from_data(rns_data)
event_df['trial_damage'] = event_df.damage.diff().fillna(0)
event_df['trial_duration'] = event_df.trial_end_time - event_df.trial_start_time
percent_missing = event_df.notnull().sum() / len(event_df)
summary_statistics = {}
summary_statistics['voice_success_rate'] = percent_missing['voice_timestamp']
if 'chunk_timestamp' in percent_missing:
    summary_statistics['chunk_success_rate'] = percent_missing['chunk_timestamp']
else:
    summary_statistics['chunk_success_rate'] = 0

# temporary fix for pilot phase where we had some incomplete data
if 'block_condition' not in event_df:
    event_df['block_condition'] = 'practice'
    event_df.loc[5:,'block_condition'] = 'voice'

event_df['spoken_difficulty_encoded'] = event_df.spoken_difficulty.replace(to_replace=['easy', 'hard'],
                                                                      value=[1, 2])

# 3. ECG Data Processing

In [6]:
post_processed_event_df = process_session_ecg(rns_data, event_df,plot_frequency=20,plot_ecg_snippet=40)

# 4. Eye Tracking Processing

In [7]:
post_processed_event_df = process_session_eye(rns_data, post_processed_event_df,detect_blink=True,plot_frequency=20, plot_eye_snippet=40)

Computed velocity exceeds threshold. Inappropriate filter setup? [2087.7 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [2566.9 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1798.8 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1811.1 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1711.0 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1991.5 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1886.0 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1862.0 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1512.8 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1002.0 > 1000.0 deg/s]
Computed velocity exceeds threshold. Inappropriate filter setup? [1054.9 > 1000.0 deg/s]
Computed velocity exc

In [10]:
post_processed_event_df

,trial_start_time,trial_end_time,ppid,session,block,number_in_block,trial,density,damage,source,...,CP4 Freq Band Power,CP2 Freq Band Power,P2 Freq Band Power,P4 Freq Band Power,P6 Freq Band Power,P8 Freq Band Power,P10 Freq Band Power,PO8 Freq Band Power,PO4 Freq Band Power,O2 Freq Band Power
0,0.000000,2534.338858,2.0,2.0,1.0,1.0,1.0,0.000000,0.000000,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2534.348858,2547.114877,2.0,2.0,1.0,2.0,2.0,0.100000,0.000000,4.0,...,5.574559e-13,7.090736e-13,6.852647e-13,5.521315e-13,1.156259e-12,1.084157e-12,1.263229e-12,1.493509e-12,1.135999e-12,1.342998e-12
2,2547.124877,2559.498461,2.0,2.0,1.0,3.0,3.0,0.300000,0.000000,4.0,...,5.336982e-13,3.514737e-13,4.585340e-13,5.627335e-13,1.184011e-12,1.812955e-12,1.158613e-12,1.631533e-12,9.437342e-13,1.161543e-12
3,2559.508461,2574.974257,2.0,2.0,1.0,4.0,4.0,0.500000,0.000000,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2574.984257,2591.728904,2.0,2.0,1.0,5.0,5.0,0.700000,0.000000,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,3709.997879,3727.054514,2.0,2.0,10.0,4.0,69.0,0.066480,68.117813,0.0,...,1.161277e-12,1.498766e-12,2.770091e-12,2.216850e-12,1.794910e-12,2.177909e-12,2.071346e-12,3.094199e-12,3.543312e-12,3.265200e-12
69,3727.064514,3743.615957,2.0,2.0,10.0,5.0,70.0,0.254417,68.117813,0.0,...,1.180116e-12,1.363976e-12,1.574226e-12,1.327388e-12,1.297830e-12,1.403529e-12,1.479037e-12,1.786509e-12,1.059614e-12,1.606240e-12
70,3743.625957,3760.319535,2.0,2.0,10.0,6.0,71.0,0.757396,68.117813,0.0,...,2.769577e-12,1.815771e-12,2.515022e-12,3.202859e-12,3.049063e-12,2.392105e-12,1.690180e-12,2.461874e-12,3.134016e-12,2.946121e-12
71,3760.329535,3776.698323,2.0,2.0,10.0,7.0,72.0,0.594553,68.117813,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 5. EEG

In [9]:
post_processed_event_df, epochs, events, event_dict, info, reject_log, ica = process_session_eeg(rns_data, post_processed_event_df,
                                                                    event_column='spoken_difficulty_encoded', run_autoreject=True, run_ica=True)

Creating RawArray with float64 data, n_channels=89, n_times=2725394
    Range : 0 ... 2725393 =      0.000 ...  1330.758 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 67585 samples (33.000 sec)

Not setting metadata
46 matching events found
Setting baseline interval to [-0.2001953125, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from pr

# 5. Motor

In [14]:
post_processed_event_df = process_session_motor(rns_data, post_processed_event_df, motor_channel='Unity_MotorInput',
                                                plot_motor_result = True, plot_motor_snippet = 30, plot_frequency = 10)


# 5. Save

In [19]:
post_processed_event_df.to_csv(f"../output/ppid_{post_processed_event_df.iloc[0].ppid}_session_{post_processed_event_df.iloc[0].session}.csv")

### Interactive PivotTable

In [20]:
from pivottablejs import pivot_ui
pivot_ui(post_processed_event_df, outfile_path=f"../output/post_processed_event_df.html");